In [1]:
"""Required imports"""
import json
import urllib

from bs4 import BeautifulSoup

In [ ]:
# We have to generate the webpage dynamically since it's mostly JS.
from contextlib import closing
from selenium.webdriver import Firefox # pip install selenium
from selenium.webdriver.support.ui import WebDriverWait

In [4]:
# Good reads data pulling.
def get_shelf(shelf_name):
    service_url = 'https://www.goodreads.com/shelf/show/'
    url = service_url + '?' + shelf_name
    response = urllib.urlopen(url).read()
    
    return response

In [5]:
get_shelf("fiction")

'<!DOCTYPE html>\n<html class="desktop">\n\n<head>\n<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"d81cb9ea53","applicationID":"4008173","transactionName":"JVteFRNcXwpRSxxKXwNYVk4SW1wRa0paXlkDUG8OFEc=","queueTime":1,"applicationTime":199,"agent":""}</script>\n<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"Ug8DUVVADAMAXFBVBA=="};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<e.length;o++)r(e[o]);return r}({1:[function(t,n,e){function r(t){try{s.console&&console.log(t)}catch(n){}}var o,i=t("ee"),a=t(15),s={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(s.console=!0,o.indexOf("dev")!==-1&&(s.dev=